In [1]:
import numpy as np

def cal_pop_fitness(equation_inputs, pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function caulcuates the sum of products between each input and its corresponding weight.
    fitness = np.sum(pop*equation_inputs, axis=1)
    return fitness

def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = np.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = np.where(fitness == np.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = np.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = np.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover


## Find the parameters that maximize Y

The following equation Y(x1, x2, ..., x6) = w1x1 + w2x2 + ... + w6x6 has inputs and weights. We want to find the parameters (weights, e.g., w1, ..., w6) that maximmize Y.

In [2]:
# Inputs of the equation. (x1, x2, ..., x6)
equation_inputs = [4,-2,3.5,5,-11,-4.7]

# Number of the weights we are looking to optimize.
num_weights = 6

#### Define initial population
The population is N sets of values of the weights. These sets are called chromosomes. (The size of the chromosome is then the number of weights. Each of the 6, for this example, values are called genes). The chromosomes are also called solutions.

In [9]:
# Based on number of weights, each chromosome (solution) in the population will have 6 genes (1 gene per weight)
# The number of solutions per population has no set value it needs to take. 
sol_per_pop = 8

# Defining population size.
# The population will have sol_per_pop chromosome where each chromosome has num_weights genes.
pop_size = (sol_per_pop, num_weights) 

# Creating the initial population.
new_population = np.random.uniform(low=-4., high=4., size=pop_size)



In [4]:
num_generations = 4
num_parents_mating = 4

for generation in range(num_generations):
    # Measuring the fitness of each chromosome in the population.
    # This is just Y = x1w1 + ... x6w6
    fitness = cal_pop_fitness(equation_inputs, new_population)
    
    # Selecting the best parents in the population for mating.
    parents = select_mating_pool(new_population, fitness, 
                                       num_parents_mating)
    
    # Generating next generation using crossover.
    offspring_crossover = crossover(parents,
                                        offspring_size=(pop_size[0]-parents.shape[0], num_weights))

    # Adding some variations to the offsrping using mutation. (It's currently hitting the 4th gene)
    offspring_mutation = mutation(offspring_crossover)
    
    # Creating the new population based on the parents and offspring.
    # We keep the parents just in case all the randomly generated offspring are worse. This prevents us from going backwards.
    # 4 parents, 4 offspring
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation
    
    # The best result in the current iteration.
    print("Best result : ", np.max(np.sum(new_population*equation_inputs, axis=1)))

# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = cal_pop_fitness(equation_inputs, new_population)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = np.where(fitness == np.max(fitness))

# print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx])
    
    
    
    

Best result :  70.56022128860512
Best result :  70.56022128860512
Best result :  83.59270694824139
Best result :  88.63665405806407
Best solution fitness :  [88.63665406]


In [8]:
equation_inputs = [4,-2,3.5,5,-11,-4.7]
new_population = np.random.uniform(low=-4., high=4., size=pop_size)
print(new_population.shape)
fitness = cal_pop_fitness(equation_inputs, new_population)


(8, 6)
